In [1]:
import cv2
import mediapipe as mp
import numpy as np
from IPython.display import Video
import os

In [2]:
from win32api import GetSystemMetrics
w =  GetSystemMetrics(0)
h = GetSystemMetrics(1)
print("Width =", GetSystemMetrics(0))
print("Height =", GetSystemMetrics(1))

Width = 1280
Height = 720


In [4]:

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose

mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils

numPoses = 0

cap = cv2.VideoCapture(0) # video capture source camera (Here webcam of laptop) 

while True: 

    ret,frame = cap.read() # return a single frame in variable `frame`

    # Create a MediaPipe `Pose` object
    with mp_pose.Pose(static_image_mode=True, 
            model_complexity=2,
                    enable_segmentation=True) as pose:
            
        # Read the file in and get dims
        #image = cv2.imread(frame)
        #display(image)

        # Convert the BGR image to RGB and then process with the `Pose` object.
        results = pose.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
    # Copy the image
    segmented_image = frame.copy()

    # Probability threshold in [0, 1] that says how "tight" to make the segmentation. Greater value => tighter.
    tightness = 0.3

    # Stack the segmentation mask for 3 RGB channels, and then create a filter for which pixels to keep

    if np.stack((results.segmentation_mask,) * 3, axis=-1).any() == None: 
        continue

    condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > tightness

    # Creates a black background image
    bg_image = np.zeros(frame.shape, dtype=np.uint8)
    figure = np.zeros(frame.shape, dtype=np.uint8)

    # Can change the color of this background by specifying (0-255) RGB values. We choose green-screen green.
    bg_image[:] = [255, 255, 255]
    figure[:] = [0, 0, 0]

    # For every pixel location, display the corresponding pixel from the original imgae if the condition in our filter is met (i.e. the probability of being part of the object is above the inclusiogn threshold), or else display corresponding pixel from the background array (i.e. green)
    segmented_image = np.where(condition, figure, bg_image)
    segmented_image = cv2.resize(segmented_image, (w,h))
    
    cv2.imshow("pose", segmented_image)
    path = "C:\Computer\Programming\opencv\c\poses"
    if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            cap = None
            break

    if cv2.waitKey(1) & 0xFF == ord('y'): #save on pressing 'y' 

        if numPoses < 1: 
            numPoses += 1

            filename = f"pose{numPoses}.jpg"

            cv2.imwrite(os.path.join(path , filename), segmented_image)

        else: 
            cv2.destroyAllWindows()
            cap = None
            break